In [18]:
!pip install apache-tvm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import tvm
from tvm import relay
from tvm.relay.testing import mlp
import numpy as np

In [20]:
target="llvm"
dtype="float32"

input_shape = (1, 1, 28, 28)
tvm_mod, params = relay.testing.mlp.get_workload(batch_size=input_shape[0], dtype=dtype)

In [21]:
with relay.build_config(opt_level=3):
	graph, lib, params = relay.build(tvm_mod,  target=target, params=params)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: legacy graph executor behavior of producing json / lib / params will be removed in the next release. Please see documents of tvm.contrib.graph_executor.GraphModule for the  new recommended usage.
  


In [22]:
print(lib.get_source())

; ModuleID = 'TVMMod'
source_filename = "TVMMod"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

%0 = type { i32*, i32 }
%1 = type { i8*, %2, i32, %3, i64*, i64*, i64 }
%2 = type { i32, i32 }
%3 = type { i8, i8, i16 }
%closure_loop_parallel_ax1.outer.ax0.outer.fused = type { i8*, i8*, i8*, i8* }
%closure_loop_parallel_ax1.outer.ax0.outer.fused.0 = type { i8*, i8*, i8*, i8* }
%closure_loop_parallel_ax1.outer.ax0.outer.fused.1 = type { i8*, i8*, i8*, i8* }

@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global void (i8*)* null, align 8
@__TVMBackendParallelLaunch = linkonce dllexport local_unnamed_addr global i32 (i32 (i32, %0*, i8*)*, i8*, i32)* null, align 8
@.str = private constant [90 x i8] c"Assert fail: (num_args == 2), tvmgen_default_fused_nn_batch_flatten: num_args should be 2\00", align 1
@.str.1 = private constant [209 x i8] c"Assert fail: ((((arg.placeholder.code == 3) || (a

In [23]:
print(lib.get_source("asm"))

	.text
	.file	"TVMMod"
	.section	.rodata.cst16,"aM",@progbits,16
	.p2align	4
.LCPI0_0:
	.long	784
	.long	784
	.long	28
	.long	1
	.text
	.globl	tvmgen_default_fused_nn_batch_flatten
	.p2align	4, 0x90
	.type	tvmgen_default_fused_nn_batch_flatten,@function
tvmgen_default_fused_nn_batch_flatten:
.Lfunc_begin0:
	.cfi_startproc
	pushq	%rax
	.cfi_def_cfa_offset 16
	cmpl	$2, %edx
	jne	.LBB0_1
	movl	(%rsi), %eax
	cmpl	$13, %eax
	ja	.LBB0_5
	movl	$8344, %ecx
	btl	%eax, %ecx
	jae	.LBB0_5
	movl	4(%rsi), %eax
	cmpl	$13, %eax
	ja	.LBB0_8
	movl	$8344, %ecx
	btl	%eax, %ecx
	jae	.LBB0_8
	movq	(%rdi), %rdx
	cmpl	$4, 16(%rdx)
	jne	.LBB0_10
	cmpw	$1, 22(%rdx)
	jne	.LBB0_14
	cmpb	$32, 21(%rdx)
	jne	.LBB0_14
	cmpb	$2, 20(%rdx)
	jne	.LBB0_14
	movq	24(%rdx), %rax
	cmpl	$1, (%rax)
	jne	.LBB0_16
	cmpl	$1, 8(%rax)
	jne	.LBB0_18
	cmpl	$28, 16(%rax)
	jne	.LBB0_20
	cmpl	$28, 24(%rax)
	jne	.LBB0_22
	movq	8(%rdi), %rax
	movq	(%rdx), %rsi
	movq	32(%rdx), %rcx
	movl	12(%rdx), %r8d
	movq	(%rax), %rdi
	movq	24(%rax), %r9

In [24]:
from tvm.contrib.debugger import debug_runtime as graph_executor
dev = tvm.device(target, 0)
m = graph_executor.create(graph, lib, dev)
	
m.set_input(**params)
m.set_input("data", np.random.normal(size=input_shape).astype(dtype))

In [25]:
m.run(number=10)

Node Name                                                 Ops                                                       Time(us)  Time(%)  Shape     Inputs  Outputs  Measurements(us)  
---------                                                 ---                                                       --------  -------  -----     ------  -------  ----------------  
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    29.365    78.543   (1, 128)  3       1        [29.365]          
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  4.216     11.276   (1, 64)   3       1        [4.216]           
tvmgen_default_fused_nn_contrib_dense_pack_add            tvmgen_default_fused_nn_contrib_dense_pack_add            2.148     5.746    (1, 10)   3       1        [2.148]           
tvmgen_default_fused_nn_softmax                           tvmgen_default_fused_nn_softmax      

In [26]:
m.run(number=10, repeat=10)

Node Name                                                 Ops                                                       Time(us)  Time(%)  Shape     Inputs  Outputs  Measurements(us)                                                                 
---------                                                 ---                                                       --------  -------  -----     ------  -------  ----------------                                                                 
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    23.362    80.554   (1, 128)  3       1        [28.812, 28.71, 25.218, 25.036, 22.287, 22.015, 20.525, 20.441, 20.218, 20.354]  
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  2.95      10.172   (1, 64)   3       1        [3.428, 3.492, 3.509, 3.435, 3.323, 2.557, 2.528, 2.338, 2.252, 2.639]           
tvmgen_default_fused_nn_

In [27]:
m.run(repeat=10, min_repeat_ms=1)

Node Name                                                 Ops                                                       Time(us)  Time(%)  Shape     Inputs  Outputs  Measurements(us)                                                                 
---------                                                 ---                                                       --------  -------  -----     ------  -------  ----------------                                                                 
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    23.844    78.977   (1, 128)  3       1        [24.496, 24.204, 25.438, 24.969, 25.042, 25.986, 24.68, 22.197, 21.914, 19.517]  
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  2.931     9.709    (1, 64)   3       1        [3.158, 3.183, 3.163, 3.189, 2.53, 2.488, 2.413, 2.351, 3.505, 3.333]            
tvmgen_default_fused_nn_

In [28]:
m.run(repeat=10, min_repeat_ms=1, cooldown_interval_ms=1)

Node Name                                                 Ops                                                       Time(us)  Time(%)  Shape     Inputs  Outputs  Measurements(us)                                                                  
---------                                                 ---                                                       --------  -------  -----     ------  -------  ----------------                                                                  
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu    21.737    80.912   (1, 128)  3       1        [26.713, 24.033, 20.306, 21.607, 20.723, 20.882, 20.719, 20.035, 21.882, 20.475]  
tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  tvmgen_default_fused_nn_contrib_dense_pack_add_nn_relu_1  2.699     10.047   (1, 64)   3       1        [5.067, 2.686, 2.461, 2.303, 2.502, 2.748, 2.316, 2.266, 2.396, 2.248]            
tvmgen_default_fused

In [29]:
r = m.run_individual(number=10)
print(r)

[[0.0], [6.635e-07], [0.0], [0.0], [2.43551e-05], [0.0], [0.0], [3.1652e-06], [0.0], [0.0], [1.5798e-06], [8.293999999999998e-07]]


In [30]:
ri = m.run_individual_node(index=0, number=10)
print(ri)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   0.0000       0.0000       0.0000       0.0000       0.0000   
               


In [31]:
ri = m.run_individual_node(index=7, number=10)
print(ri)

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   0.0029       0.0029       0.0029       0.0029       0.0000   
               
